In [1]:
import pandas as pd
from Bio import SeqIO 
import os,sys
import sgRNA_utils.sgRNA_primer_util as su
import module.sequencing_primer as sr
import module.from_gbFile_to_seq as fq
import module.parser_input_to_df as pf
import module.product_and_decorate_editingSeq as p_d_seq
import module.order as order
import warnings   
warnings.filterwarnings('ignore')  
import configparser
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
from sgRNA_utils.sgRNA_primer_config import config 
from loguru import logger

input_path = '/home/yanghe/program/edit_sequence_design/input/config.json'

In [12]:
#uha_dha_primer
def extract_uha_dha_primer(info_input_df, sgRNA):

    primer_template_for_u_d_ha_df = p_d_seq.create_primer_template(info_input_df,sgRNA)
    uha_dha_primer_df = p_d_seq.design_primer(primer_template_for_u_d_ha_df,'Name_Region','primer_template','u_d')
    uha_dha_primer_df = uha_dha_primer_df.rename(columns={'Region':'id'})
    uha_dha_primer_df = uha_dha_primer_df.join(uha_dha_primer_df.id.str.split(';',expand=True).rename(columns = {0:'Name',1:'Region',2:'Type'})).drop(columns='id')
    
    return uha_dha_primer_df

#uha_dha
def extract_uha_dha(info_input_df,uha_dha_primer_df,sgRNA):
    #整合进突变信息
    info_df = info_input_df[['name','region','seq_altered','type','ref','strand']].rename(columns={'name':'Name','region':'Region'})
    info_df.seq_altered.fillna('',inplace=True)
    uha_dha_info_primer_df = pd.merge(info_df,uha_dha_primer_df,on=['Name','Region'])

    #提取源生同源臂
    uha_dha_df = p_d_seq.create_uha_dha_df(uha_dha_primer_df) 
    #合并突变信息
    uha_dha_df = pd.merge(uha_dha_df,info_df,on=['Name','Region'])
    uha_dha_sgRNA_df = pd.merge(uha_dha_df,sgRNA,on=['Name','Region'],how='inner')

    return uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df,info_df
def two_plasmid_system_n20_enzyme_cut_seq(no_ccdb_uha_dha_sgRNA_df,promoter_seq,enzyme_df,enzyme_name):

    sgRNA_enzyme_df = enzyme_df[enzyme_df['name']==enzyme_name]
    cut_seq_len = sgRNA_enzyme_df.loc[0,'cut_seq_len']
  

    #N20片段引物退火获得
    temp_sgRNA_df = no_ccdb_uha_dha_sgRNA_df[['Name', 'Region','Target sequence', 'promoter_N20_terminator']]
    temp_sgRNA_df['Region'] = temp_sgRNA_df['Name'] +';'+ temp_sgRNA_df['Region']
    temp_sgRNA_df.drop(columns='Name',inplace=True)
    enzymeCutSeq_and_N20_df = p_d_seq.create_enzymeCutSeq_and_N20(temp_sgRNA_df, promoter_seq, enzyme_cut_len=cut_seq_len)
    #取出重要列
    enzymeCutSeq_and_N20_df = enzymeCutSeq_and_N20_df.rename(columns = {"Region":"ID"})
    enzymeCutSeq_and_N20_df = enzymeCutSeq_and_N20_df[[ "ID","Target sequence","enzymeCutSeq_and_N20"  ]]

    return enzymeCutSeq_and_N20_df

def region_2_distance(sgRNA_plasmid_seq_len, sgRNA_region_json, first_primer_start_position):

    distance_dict = {}
    for k,v in sgRNA_region_json.items():
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        if first  > first_primer_start_position:
            distance = first - first_primer_start_position
        else:
            distance = sgRNA_plasmid_seq_len - first_primer_start_position + first
        distance_dict.update({distance:v})
    sorted_distance = sorted(distance_dict.keys())

    last_distance_dict = {}
    for i in range(len(sorted_distance)):
        v = distance_dict[sorted_distance[i]]
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        new_len = last - first

        if i != 0:
            v = distance_dict[sorted_distance[i-1]]
            arr = v.split(',')
            first = int(arr[0])
            last = int(arr[1])
            old_len = last - first
            min_distance = sorted_distance[i] - sorted_distance[i-1] -old_len
        else:
            min_distance = sorted_distance[i]  

        max_distance = min_distance + new_len

        last_distance_dict.update({f'primer{i+1}':(min_distance, max_distance)})

    print(last_distance_dict)

    return last_distance_dict

# 数据预处理

In [3]:
with open(input_path, "r") as f:
        data = json.load(f)
chopchop_input = data['chopchop_input']
    # enzyme_path = parent_base_path +'/'+ data['enzyme_path']
sgRNA_result_path = data['sgRNA_result_path']
one_plasmid_file_path = data['one_plasmid_file_path']
no_ccdb_plasmid = data['no_ccdb_plasmid']
no_sgRNA_plasmid = data['no_sgRNA_plasmid']

 #plasmid label
ccdb_label = data['plasmid_label']['ccdb_label']
promoter_terminator_label = data['plasmid_label']['promoter_terminator_label']
n_20_label = data['plasmid_label']['n_20_label']

#primer
sgRNA_primer_json = data['sgRNA_primer_json']
ccdb_primer_json = data['ccdb_primer_json']

sgRNA_region_json = data['sgRNA_region_json']
ccdb_region_json = data['ccdb_region_json']

#配置引物参数
config.S_GLOBAL_ARGS = data['S_GLOBAL_ARGS']
config.Q_ARGS = data['Q_ARGS']

   #配置输出参数
output = data['edit_sequence_design_workdir']
if not os.path.exists(output):
    os.makedirs(output)
# 1.read 编辑序列信息
info_input_df = pd.read_csv(chopchop_input)  

enzyme = data['enzyme']
enzyme_df = pd.DataFrame([enzyme])
enzyme_name =data['enzyme']['name']

In [4]:
# 3.提取用户选择的sgRNA
sgRNA = p_d_seq.extract_sgRNA_from_chopchop(sgRNA_result_path)

In [5]:
# 4.设计源生同源臂引物
uha_dha_primer_df = extract_uha_dha_primer(info_input_df, sgRNA)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{'PRIMER_LEFT_EXPLAIN': 'considered 8, ok 8', 'PRIMER_RIGHT_EXPLAIN': 'considered 408, low tm 128, not in any ok right region 172, ok 108', 'PRIMER_PAIR_EXPLAIN': 'considered 1, ok 1', 'PRIMER_LEFT_NUM_RETURNED': 1, 'PRIMER_RIGHT_NUM_RETURNED': 1, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 1, 'PRIMER_PAIR_0_PENALTY': 10.372888476322885, 'PRIMER_LEFT_0_PENALTY': 3.394434015295303, 'PRIMER_RIGHT_0_PENALTY': 6.978454461027582, 'PRIMER_LEFT_0_SEQUENCE': 'TTATTTCCGTGCGCGTTGGTGA', 'PRIMER_RIGHT_0_SEQUENCE': 'GAAGCCATAGAAGCCTGCAT', 'PRIMER_LEFT_0': (0, 22), 'PRIMER_RIGHT_0': (1049, 20), 'PRIMER_LEFT_0_TM': 63.6055659847047, 'PRIMER_RIGHT_0_TM': 58.02154553897242, 'PRIMER_LEFT_0_GC_PERCENT': 50.0, 'PRIMER_RIGHT_0_GC_PERCENT': 50.0, 'PRIMER_LEFT_0_SELF_ANY_TH': 19.610968059260017, 'PRIMER_RIGHT_0_SELF_ANY_TH': 0.0, 'PRIMER_LEFT_0_SELF_END_TH': 5.11952728907761, 'PRIMER_RIGHT_0_SELF_END_TH': 0.0, 'PRIMER_LEFT_0_HAIRPIN_TH': 36.149651650343515, 'PRIM

In [6]:
# 5.提取同源臂
uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df, info_df = extract_uha_dha(info_input_df,uha_dha_primer_df,sgRNA)

Index(['primer_f_seq_(5'-3')', 'primer_r_seq_(5'-3')', 'primer_f_Tm',
       'primer_r_Tm', 'product_size', 'product_value', 'Name', 'Region',
       'Type'],
      dtype='object')


# 双质粒设计

## 构建两个新质粒

In [7]:
no_ccdb_uha_dha_sgRNA_df,\
sgRNA_plasmid_backbone,\
promoter_seq,\
terminator_seq,\
sgRNA_promoter_terminator= p_d_seq.create_new_plasmid(no_ccdb_plasmid,uha_dha_sgRNA_df.copy(),ccdb_label, promoter_terminator_label, n_20_label)


no_sgRNA_uha_dha_ccdb_df,\
ccdB_plasmid_backbone,\
ccdB_promoter_terminator_up_seq  = p_d_seq.create_new_plasmid(no_sgRNA_plasmid,uha_dha_sgRNA_df.copy(),ccdb_label, promoter_terminator_label, n_20_label)

-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


In [8]:
#酶切退火方式获得质粒和n20
enzymeCutSeq_and_N20_df = two_plasmid_system_n20_enzyme_cut_seq(no_ccdb_uha_dha_sgRNA_df,promoter_seq,enzyme_df,enzyme_name)

In [12]:
#选择引物设计类型
#质粒引物的设计类型：1---用户指定范围，2----无需用户指定范围，3----用户指定额外引物

In [9]:
plasmid_primer_desgin_type = 1

In [10]:
#将sgRNA质粒分割成：promoter_terminator_up_seq、promoter_seq、n20_coordinate_seq、terminator_seq、promoter_terminator_down_seq
sgRNA_plasmid_seq, sgRNA_plasmid_region_seq = p_d_seq.plasmid_region_division_by_labels(no_ccdb_plasmid,
                                                                                        ccdb_label,
                                                                                        promoter_terminator_label,
                                                                                        n_20_label)
#将ccdb质粒分割成：ccdb_up_seq、ccdb、ccdb_down_seq
ccdb_plasmid_seq, ccdb_plasmid_region_seq = p_d_seq.plasmid_region_division_by_labels(no_sgRNA_plasmid,
                                                                                      ccdb_label,
                                                                                      promoter_terminator_label,
                                                                                      n_20_label)

-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


In [15]:
#针对sgRNA质粒的操作：
#确定第一条引物的位置在质粒上的位置：先找到启动子终止子的绝对位置，再确定相对位置，再确定绝对位置
promoter_terminator_seq = sgRNA_plasmid_region_seq['promoter_seq']+sgRNA_plasmid_region_seq['n20_coordinate_seq']+sgRNA_plasmid_region_seq['terminator_seq']
promoter_terminator_start = sgRNA_plasmid_seq.find(promoter_terminator_seq)
promoter_terminator_end = promoter_terminator_start + len(promoter_terminator_seq)
first_primer_position_in_promoter_terminator = promoter_terminator_seq.find(sgRNA_plasmid_region_seq['terminator_seq'])
first_primer_start_position = promoter_terminator_start + first_primer_position_in_promoter_terminator

#将用户提供的质粒设计区域转换成，各个区域之间的最小，最大距离
sgRNA_plasmid_seq_len = len(sgRNA_plasmid_seq)
sgRNA_distance_dict = region_2_distance(sgRNA_plasmid_seq_len, sgRNA_region_json,first_primer_start_position)

#针对区域设计引物，特殊处理第一条和最后一条引物
sgRNA_plasmid_primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(first_primer_start_position, sgRNA_plasmid_seq, sgRNA_distance_dict, 20)
sgRNA_plasmid_primer_df =  su.result_primer_list_to_df(sgRNA_plasmid_primer_result_list)


#针对ccdb载体质粒的操作：
ccdb_start = ccdb_plasmid_seq.find(ccdb_plasmid_region_seq['ccdb'])
ccdb_end = ccdb_start + len(ccdb_plasmid_region_seq['ccdb'])
first_primer_start_position = ccdb_end
ccdb_plasmid_seq_len = len(ccdb_plasmid_seq)
ccdb_distance_dict = region_2_distance(ccdb_plasmid_seq_len, ccdb_region_json,first_primer_start_position)
ccdb_plasmid_primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(first_primer_start_position,ccdb_plasmid_seq, ccdb_distance_dict,len(ccdb_plasmid_region_seq['ccdb']))
ccdb_plasmid_primer_df = su.result_primer_list_to_df(ccdb_plasmid_primer_result_list)

{'primer1': (3001, 3200), 'primer2': (3002, 3200)}
3770 8716 hdsfjkgfhjgjghfj
{'primer1': (3001, 3200), 'primer2': (3001, 3200)}
570 5363 hdsfjkgfhjgjghfj


In [21]:
#sgRNA质粒引物加接头
sgRNA_plasmid_primer_joint_df = p_d_seq.sgRNA_sgRNAprimer_merge(uha_dha_sgRNA_df, sgRNA_plasmid_primer_df)
sgRNA_primers_sum=len(sgRNA_plasmid_primer_df)
sgRNA_plasmid_primer_joint_df = p_d_seq.add_joint_plasmid_primer(enzyme_df,
                                                                 enzyme_name,
                                                                 sgRNA_plasmid_primer_joint_df,
                                                                 sgRNA_primers_sum,
                                                                 primer_type='sgRNA')

primer_r_seq_(5'-3')_3
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC
AACTATTTATCCAGTTGGTACAAAC AGTT
GAAGATAACGAACAAAAACAATTGT CTTC


In [22]:
sgRNA_plasmid_primer_joint_df

,Region,Rev Target sequence,primer_f_seq_(5'-3')_1,primer_f_seq_(5'-3')_2,primer_f_seq_(5'-3')_3,primer_r_seq_(5'-3')_1,primer_r_seq_(5'-3')_2,primer_r_seq_(5'-3')_3,product_value_1,product_value_2,product_value_3
0,153019_ecoil_ybeL_ins;BA000036.3:153019-153020,AGAAAACTTAAAAGCGAACG,CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTAGAAAACTTAAAAGCGAACGTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
1,Cgl0006_1176_G_A_sub;BA000036.3:6529-6530,CGGTTATTTCCGTGCGCGTT,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTCGGTTATTTCCGTGCGCGTTTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
2,Cgl0141_cds_del;BA000036.3:153222-155613,CGGTGTGTTAATCCGCCAGC,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTCGGTGTGTTAATCCGCCAGCTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
3,Cgl0591_-1_Ppgk_promoter_ins;BA000036.3:604469...,CTGCGCGTAACCTGGGAAGC,CCAGGTCTCAGCAGGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTCTGCGCGTAACCTGGGAAGCTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
4,Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378,CGGAGAAGGTGTACTTCTCG,CCAGGTCTCATCCGGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTCGGAGAAGGTGTACTTCTCGTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
5,Cgl1386_327_18to15_sub;BA000036.3:1454441-1454459,TAGCCACCTTCTGATTCTTC,CCAGGTCTCAGCTAGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTTAGCCACCTTCTGATTCTTCTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
6,Cgl1436_1113_CAA_del;BA000036.3:1514089-1514092,TCCGTCTGCAAAGTGGTGGT,CCAGGTCTCACGGAGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTTCCGTCTGCAAAGTGGTGGTTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
7,Cgl1790_1647_TCC_sub;BA000036.3:1899017-1899020,TAATCTTATTGACTTCTTCT,CCAGGTCTCAATTAGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,CCAGGTCTCAGAAGATAACGAACAAAAACAATTGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,GGTCCAGAGTCTTCTGGACTACCCTTCAACTTTTCA,GGTCCAGAGTTAATCTTATTGACTTCTTCTTGAATTACACTGTACC...,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...,AACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGA...
8,Cgl2342_213_GCA_ins;BA000036.3:2484874-2484875,GAAATTTCCGTCATGCCGAC,CCAGGTCTCATTTCGTTTTAGAGCTAGAAATAGCAAGT,CCAGGTCTCAAACTATTTATCCAGTTGGTACA

In [23]:
asd ={

        "chopchop_input": "./input/info_input.csv",
        "sgRNA_result_path": "./input/all_result.csv",
        "edit_sequence_design_workdir":"./output/edit_sequence_design/",

        "one_plasmid_file_path": "./input/pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_ccdb_plasmid":"./input/no-ccdb-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_sgRNA_plasmid":"./input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        
        "plasmid_label":{
            "ccdb_label":"ccdB",
            "promoter_terminator_label":"gRNA",
            "n_20_label":"N20"
        },
        
        "sgRNA_primer_json":{
            "primer3":"AACTATTTATCCAGTTGGTACAAAC",
            "primer5":"GAAGATAACGAACAAAAACAATTGT"
        },
        "ccdb_primer_json":{
            "primer3":"AACTGATTCAGTCTGATTTCGCGGT",
            "primer5":"CCCTCTAATCGAAACTAATGGGGA"
        },
    
        "sgRNA_region_json":{
           
            
        },
        
        "ccdb_region_json":{
            "region1":"8364,8563",
            "region2":"376,575"
        },
        
        "S_GLOBAL_ARGS":{
            "PRIMER_OPT_SIZE": 20,   
            "PRIMER_MIN_SIZE": 18,
            "PRIMER_MAX_SIZE": 25,
            "PRIMER_OPT_TM": 65.0,
            "PRIMER_MIN_TM": 55.0,
            "PRIMER_MAX_TM": 75.0,
            "PRIMER_MIN_GC": 20.0,
            "PRIMER_MAX_GC": 80.0
        },
        
        "Q_ARGS":{
            "PRIMER_OPT_SIZE": 20,
            "PRIMER_MIN_SIZE": 18,   
            "PRIMER_MAX_SIZE": 25,
            "PRIMER_OPT_TM": 65.0,
            "PRIMER_MIN_TM": 55.0,
            "PRIMER_MAX_TM": 75.0,    
            "PRIMER_MIN_GC": 20.0,
            "PRIMER_MAX_GC": 80.0
        },
    
        "enzyme":{
            "name":"BsaI",
            "protective_base":"CCA",
            "recognition_seq":"GGTCTC",
            "cut_seq_len":4,
            "gap_len":1    
        }

    }
    

In [25]:
asd['sgRNA_region_json'] == {}

True

3